In [1]:
import pandas as pd
import gzip

# Đọc 2 file gzipped
df_basics = pd.read_csv('title.basics.tsv', sep='\t', low_memory=False)
df_ratings = pd.read_csv('title.ratings.tsv', sep='\t', low_memory=False)
df_crew = pd.read_csv('title.crew.tsv', sep='\t', low_memory=False)

In [2]:
# Gộp 3 file thông qua cột "tconst"
merged_df = df_basics.merge(df_ratings, on='tconst', how='inner')  # Inner join (keeps all rows)
merged_df = merged_df.merge(df_crew, on='tconst', how='inner')  # Merge with crew data

# Chuyển file thành định dạng csv
merged_df.to_csv('merged_data.tsv', sep='\t', index=False)  # Without index column

# In ra 
print(merged_df)

            tconst  titleType                primaryTitle  \
0        tt0000001      short                  Carmencita   
1        tt0000002      short      Le clown et ses chiens   
2        tt0000003      short              Pauvre Pierrot   
3        tt0000004      short                 Un bon bock   
4        tt0000005      short            Blacksmith Scene   
...            ...        ...                         ...   
1426275  tt9916730      movie                      6 Gunn   
1426276  tt9916766  tvEpisode              Episode #10.15   
1426277  tt9916778  tvEpisode                      Escape   
1426278  tt9916840  tvEpisode  Horrid Henry's Comic Caper   
1426279  tt9916880  tvEpisode   Horrid Henry Knows It All   

                      originalTitle isAdult startYear endYear runtimeMinutes  \
0                        Carmencita       0      1894      \N              1   
1            Le clown et ses chiens       0      1892      \N              5   
2                    Pauvre

In [3]:
# Đếm số lượng dòng có ít nhất 1 giá trị null
missing_row_count = merged_df.isnull().any(axis=1).sum()
print(missing_row_count)

3


In [4]:
# gán merged_df vào 1 dataframe khác và sử dụng hàm dropna
filtered_df = merged_df.dropna()
# gán lại dataframe vào merged_df ban đầu 
merged_df = filtered_df
# in ra số lượng dòng merged_df
print(len(merged_df))

1426277


In [5]:
import pandas as pd
import numpy as np
from scipy import stats

# Hàm tìm outliers sử dụng z-scores
def identify_outliers_zscore(df, threshold=3):
    # chọn ra các cột có kiểu dữ liệu là số
  numerical_cols = df.select_dtypes(include=np.number) 
    #khởi tạo 1 dataframe rỗng
  outliers_df = pd.DataFrame() 
    # duyệt qua từng cột
  for col in numerical_cols:
      # tính absolute z-scores
    z_scores = np.abs(stats.zscore(df[col])) 
      # Lấy dòng với outlier trong từng cột
    outlier_rows = df[z_scores > threshold].index  
      # Thêm outlier vào dataframe
    outliers_df = pd.concat([outliers_df, df.loc[outlier_rows]], ignore_index=True)  
  return outliers_df.drop_duplicates()  
    # Xoá trùng (duplicate) nếu tồn tại


In [10]:
outlier_row_df = identify_outliers_zscore(merged_df.copy(), 3)
print(outlier_row_df)

          tconst  titleType                          primaryTitle  \
0      tt0001036      short                    Sherlock Holmes IV   
1      tt0002739      movie                                Carmen   
2      tt0003817      movie                                   Dan   
3      tt0004381      movie        The Mystery of the Poison Pool   
4      tt0004398      movie                   The Next in Command   
...          ...        ...                                   ...   
20481  tt9844522      movie  Escape Room: Tournament of Champions   
20482  tt9866072      movie                              Holidate   
20483  tt9893250      movie                          I Care a Lot   
20484  tt9898836  tvEpisode                          Perfect Game   
20485  tt9906260  tvEpisode                                  Hero   

                              originalTitle isAdult startYear endYear  \
0                        Sherlock Holmes IV       0      1909      \N   
1                        

In [11]:
print(merged_df.shape)

(1426277, 13)


In [15]:
# Drops rows based on indices in outlier_rows_df
temp_df = merged_df.drop(outlier_row_df.index)  

In [17]:
merged_df = temp_df
print(merged_df.shape)

(1405809, 13)
